In [2]:
!pip install python-igraph
!pip install pycairo
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import csv 
import pandas as pd
from igraph import *
import cairo

In [4]:
path = "samples_thayna/captura_2.4G.csv" # Path to the .csv file

with open(path) as file:
    table = csv.reader(file)
    rows  = list(table)[1:] # Step to remove st empty row
    aps   = []
    
    for data in rows:
        if not data: 
            break
        
        aps.append(data)
    
df = pd.DataFrame(aps[1:], columns = rows[0])
df

,BSSID,First time seen,Last time seen,channel,Speed,Privacy,Cipher,Authentication,Power,# beacons,# IV,LAN IP,ID-length,ESSID,Key
0,E8:D2:FF:96:76:0F,2022-08-11 18:09:16,2022-08-12 04:45:09,1,260,WPA2 WPA,CCMP TKIP,PSK,-36,1717,178,0. 0. 0. 0,8,AREDE 2G,
1,70:4F:57:13:7F:E1,2022-08-11 18:08:56,2022-08-12 04:44:44,7,130,WPA2,CCMP TKIP,PSK,-36,5592,180,0. 0. 0. 0,12,NET_2GEA1CD1,
2,B0:BB:E5:3D:76:56,2022-08-11 21:54:47,2022-08-12 04:44:06,6,260,WPA2,CCMP,PSK,-91,438,86,0. 0. 0. 0,17,CLARO APT 1201 5G,
3,D8:32:14:52:6A:39,2022-08-11 20:34:26,2022-08-12 04:42:58,1,130,WPA2,CCMP,PSK,-89,86,6,0. 0. 0. 0,2,PK,
4,98:1E:19:15:44:69,2022-08-11 20:31:32,2022-08-12 04:33:42,1,195,WPA2,CCMP,PSK,-36,4,0,0. 0. 0. 0,13,FATIMA NET 2G,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,EC:08:6B:88:AE:4D,2022-08-11 18:08:57,2022-08-12 04:45:11,4,130,WPA2 WPA,CCMP TKIP,PSK,-32,22362,0,0. 0. 0. 0,26,Net-Virtua-8048-Computator,
60,BC:9B:68:96:2C:8A,2022-08-11 18:08:55,2022-08-12 04:45:09,1,195,WPA2,CCMP,PSK,-30,22001,15367,0. 0. 0. 0,8,MariaA2G,
61,44:D4:53:FE:6B:24,2022-08-11 18:08:55,2022-08-12 04:45:09,1,195,WPA2 WPA,CCMP TKIP,PSK,-31,19215,1890,0. 0. 0. 0,6,MEL_2G,
62,1E:E8:29:C4:F7:F1,2022-08-11 18:09:17,2022-08-12 04:45:09,1,195,WPA2,CCMP,PSK,-29,16400,0,0. 0. 0. 0,0,,


In [52]:
graph = Graph()
nodes = len(df) + 1
graph.add_vertices(nodes)

for i in range(nodes):
    graph.vs[i]["id"] = i
    
    if i == 0:
        graph.vs[i]["label"] = "My PC"
        graph.vs[i]["color"] = "red"
        
    else:
        ap_mac   = df.iloc[i - 1, 0]
        ap_ESSID = df.iloc[i - 1, -2]
        
        graph.vs[i]["label"] = (ap_mac, ap_ESSID)
        graph.vs[i]["color"] = "blue"
        
edges = []
for i in range(nodes - 1):
    edges.append((0, i + 1))

graph.add_edges(edges)

power     = []
node_dist = []

for i in range(len(df)):
    node_dist.append(1 / (int(df.iloc[i, 8]) * -1))
    power.append((int(df.iloc[i, 8]) * -1))
    
graph.es['label']  = power
graph.es['weight'] = node_dist

In [56]:
visual_style = {}
output_name = "graph.png"

visual_style["bbox"]        = (8000,8000)
visual_style["vertex_size"] = 15
visual_style["vertex_label_size"] = 15
visual_style["edge_show_label"]   = True

my_layout = graph.layout_fruchterman_reingold(weights = graph.es['weight'])
visual_style["layout"] = my_layout

plot(graph, output_name, **visual_style)
print("Plotted")

Plotted
